In [ ]:
import tensorflow as tf
import keras
from keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam
import cv2
import os
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, roc_auc_score

In [ ]:
dataset_path = 'Datasets/um2'

In [ ]:
image_size = (32,32)
batch_size = 10

#Data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_batches = train_datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    color_mode = 'grayscale',
    class_mode='sparse',
    subset='training'
)

validation_batches = train_datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    color_mode = 'grayscale',
    class_mode='sparse',
    subset='validation'
)

test_batches = train_datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    color_mode = 'grayscale',
    class_mode='sparse',
    shuffle=False,
    subset='validation'
)

In [ ]:
train_batches.class_indices

In [ ]:
u2_characters = ['ो','ाै']

In [ ]:
def plotImages(image_arr):
    fig, axes = plt.subplots(1,5)
    axes = axes.flatten()
    for img, ax in zip(image_arr, axes):
        ax.imshow(img[:,:,0])
        ax.axis('off')
    plt.tight_layout()
    plt.show()

imgs, labels = train_batches[0]
plotImages(imgs)
for i in range(5):
  print(u2_characters[int(labels[:5][i])],end='\t')

In [ ]:
imgs.shape

In [ ]:
labels.shape

In [ ]:
model2 = Sequential()

model2.add(Conv2D(32, (5, 5), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform', input_shape = (32, 32, 1)))
model2.add(Conv2D(32, (5, 5), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform'))
model2.add(Conv2D(32, (5, 5), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform'))
model2.add(MaxPooling2D((2, 2), strides = (2, 2)))
model2.add(Dropout(0.25))

model2.add(Conv2D(64, (3, 3), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform'))
model2.add(Conv2D(64, (3, 3), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform'))
model2.add(Conv2D(64, (3, 3), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform'))
model2.add(MaxPooling2D((2, 2), strides = (2, 2)))
model2.add(Dropout(0.25))

model2.add(Flatten())

model2.add(Dense(128, activation = 'relu', kernel_initializer = 'he_uniform'))
model2.add(Dropout(0.25))

model2.add(Dense(2, activation = 'softmax'))

In [ ]:
model2.summary()

In [ ]:
model2.compile(optimizer = Adam(learning_rate = 1e-3, decay = 1e-5), 
               loss = 'sparse_categorical_crossentropy', 
               metrics = ['accuracy'])

In [ ]:
if not os.path.isdir('Models/u2_Model'):
    os.mkdir('Models/u2_Model')

In [ ]:
callbacks2 = [ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1,
                              patience = 7, min_lr = 1e-5),
             EarlyStopping(monitor = 'val_loss', patience = 9, # Patience should be larger than the one in ReduceLROnPlateau
                          min_delta = 1e-5),
             CSVLogger(os.path.join('Models/u2_Model','u_training_2.csv'), separator =',', append = True),
             ModelCheckpoint(os.path.join('Models/u2_Model', 'backup_last_model.hdf5')),
             ModelCheckpoint(os.path.join('Models/u2_Model', 'best_val_acc.hdf5'), monitor = 'val_accuracy', mode = 'max', save_best_only = True),
             ModelCheckpoint(os.path.join('Models/u2_Model', 'best_val_loss.hdf5'), monitor = 'val_loss', mode = 'min', save_best_only = True)]

In [ ]:
model2.fit(train_batches, epochs = 20, validation_data = validation_batches, callbacks = callbacks2)

In [ ]:
model2 = load_model(os.path.join('Models/u2_Model', 'best_val_loss.hdf5'))

In [ ]:
i=0
pred2=[]
num2=model2.predict(test_batches)
l=len(num2)
#print(num2)
for i in range(l):
    val=np.argmax(num2[i])
    i+=1
    pred2.append(val)

#print(pred2)

In [ ]:
u_cm2=confusion_matrix(test_batches.classes,pred2)
print(u_cm2)